# Examples Notebook


### Creating an Empty Portfolio.

Portfolio objects require a `SQLite` database corresponding to them, you must specify the path to an existing one or the path where you wish one to be created.

In [9]:
from Portfolio import Portfolio

# Here we create a Portfolio object, if the database does not exist it will be created and initalised with the correct
# table. If the database already exists it will be connected to.

path = r'database.db'
port = Portfolio(path)

### Adding Positions

 As we all want to make money on the stock market, we will have to make trades or have already made trades. In either case we want to track how well we are doing so we should add them to the portfolio.

#### Adding a single position, with a known purchase price.

The `add_position_single` method is used to add a single position to the database when we know the price we purchased at. There is currently no support for different currencies, but this will be added soon.

In [10]:
# The add_position_single method takes five arguments, all of which are required. They are as follows:
# 1. The ticker of the stock, 
# 2. The direction of the trade, either 'long' or 'short'
# 3. The number of shares purchased
# 4. The purchase_price.
# 5. The date of purchase, of the form 'YYYY-MM-DD'.

ticker = 'AAPL'
direction = 'long'
shares = 100
purchase_price = 100.0
date = '2021-01-01'

port.add_position_single(ticker, direction, shares, purchase_price, date)

#### Adding multiple positions at once, with a known purchase price.

To add multiple positions to the portfolio, we use the `add_position_df` method which takes a pandas dataframe as input.

In [11]:
import pandas as pd

# The add_position_df method takes a single argument a pandas DataFrame. The DataFrame must have the following
# columns: 'ticker', 'direction', 'shares', 'purchase_price', 'date'. The 'date' column must be of the form 'YYYY-MM-DD'.

data = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL'], 
                     'direction': ['long', 'long', 'long'],
                     'shares': [100, 200, 300], 
                     'purchase_price': [100.0, 200.0, 300.0], 
                     'purchase_date': ['2021-01-01', '2021-01-02', '2021-01-03']})

port.add_position_df(data)
print(port)

    shares  purchase_price
0      100      100.000000
1      100      100.000000
2      200      200.000000
3      300      300.000000
4      100      100.000000
5      100      100.000000
6      200      200.000000
7      300      300.000000
8      100      100.000000
9      100      100.000000
10     200      200.000000
11     300      300.000000
12     100      100.000000
13     100      100.000000
14     200      200.000000
15     300      300.000000
16     100        0.412500
17     200       26.719999
18     300        2.502503
19     100      100.000000
20     100      100.000000
21     200      200.000000
22     300      300.000000
23     100        0.412500
24     200       26.719999
25     300        2.502503
26     100      100.000000
27     100      100.000000
28     200      200.000000
29     300      300.000000
30     100        0.412500
31     200       26.719999
32     300        2.502503
33     100      100.000000
34     100      100.000000
35     200      200.000000
3

#### Adding multiple positions when only purchase date is known.

Often, we may want to backtest a signal hence we will wish to simulate trades that happened in the past. It is hence useful to have a method which can be given a trade at a certain date which will fetch the opening or closing price of the symbol, emulating that we purchased it. This is the purpose of the `add_past_position` method, which uses `yahooquery` to fetch the opening or close price on the `purchase_date`.

One current caveat of `add_past_position` it adds trades which took place on the same day, hence it takes `purchase_date` as a seperate argument.

In [12]:
# The add_position_df method takes three arguments a pandas DataFrame, purchase date and type
# which defaults to open. 
# The DataFrame must have the following columns: 'ticker', 'direction', 'shares'. 

type = 'open'
purchase_date = '2004-02-17'
data_past = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL'], 
                     'direction': ['long', 'long', 'long'],
                     'shares': [100, 200, 300]})

port.add_past_position(data_past, purchase_date, type)
print(port)

    shares  purchase_price
0      100      100.000000
1      100      100.000000
2      200      200.000000
3      300      300.000000
4      100      100.000000
5      100      100.000000
6      200      200.000000
7      300      300.000000
8      100      100.000000
9      100      100.000000
10     200      200.000000
11     300      300.000000
12     100      100.000000
13     100      100.000000
14     200      200.000000
15     300      300.000000
16     100        0.412500
17     200       26.719999
18     300        2.502503
19     100      100.000000
20     100      100.000000
21     200      200.000000
22     300      300.000000
23     100        0.412500
24     200       26.719999
25     300        2.502503
26     100      100.000000
27     100      100.000000
28     200      200.000000
29     300      300.000000
30     100        0.412500
31     200       26.719999
32     300        2.502503
33     100      100.000000
34     100      100.000000
35     200      200.000000
3

### Evaluating the portfolio

Well how much money have we made? You ask. We need methods which allow us to track how well our portfolio is performing! The first of which, is how much money we invested origionaly in our portfolio (aka when we created all of our trades.).

#### Intial Valuation.

In [13]:
# Get the inital value of the portfolio.
print(port.initalValue())

1386816.0044252872


#### Current / Past Valuation

This method allows us to be more specific, we can choose whether we want to see absolute/percentage profit. See how the portfolio has profited on a day in the past, at open or at close, or simply the current valuation of the portfolio at its current point in the day. We give an example of each below.

In [14]:
# Getting the current absolute profit of the portfolio.
print(port.absoluteProfit())

# Getting the percentage profit of at close, for a date in the past. 
print(port.percentageProfit('close', '2022-01-01'))

907374.0102231503
0.5095987744453575


### When we wish to sell after a certain amount of days.

The function `trade_sell_after` will calculate the profit of the portfolio if the trades were to be cashed out after a certain amount of days. This does not alter the portfolo. 

In [15]:
# Trade_sell_after takes the argument days which must be an integer..
print(port.trade_sell_after(1))
print(port.trade_sell_after(100))

-0.35485269756821175
-0.2519067448110928


### Metrics and Illustrations.

We will want to understand how our portolfio is doing, there are a number of ways to do that. The first Illustrates how each individual trade is performing.

In [17]:
print(port.upDown(verbose=True))
print(port.sharpe_ratio())

Percentage of positions profiting: 83.33333333333334%
            Positions that are up: [symbol                  AAPL
direction               long
shares                   100
purchase_price         100.0
purchase_date     2021-01-01
Name: 0, dtype: object, symbol                  AAPL
direction               long
shares                   100
purchase_price         100.0
purchase_date     2021-01-01
Name: 1, dtype: object, symbol                  MSFT
direction               long
shares                   200
purchase_price         200.0
purchase_date     2021-01-02
Name: 2, dtype: object, symbol                  AAPL
direction               long
shares                   100
purchase_price         100.0
purchase_date     2021-01-01
Name: 4, dtype: object, symbol                  AAPL
direction               long
shares                   100
purchase_price         100.0
purchase_date     2021-01-01
Name: 5, dtype: object, symbol                  MSFT
direction               long
shares 